In [1]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"
dump_path = data_path / 'initial_dump'

# train test split paths
train_path = data_path / 'fsm_tts/train'
test_path = data_path / 'fsm_tts/test'

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import modeling packages
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# modeling metrics
from sklearn.metrics import classification_report, confusion_matrix

changes to data generators:

    same as previous model

In [9]:
train_generator = ImageDataGenerator().flow_from_directory(str(train_path),
                                                           target_size=(150, 150),
                                                           batch_size=128,
                                                           class_mode='categorical')
test_generator = ImageDataGenerator().flow_from_directory(str(test_path),
                                                          target_size=(150, 150),
                                                          batch_size=10,
                                                          class_mode='categorical')

Found 7858 images belonging to 7 classes.
Found 72 images belonging to 7 classes.


changes to model architecture:
    
    same as last model

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(7, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

changes to fit:
    
    same as last model

In [11]:
model.fit(
        train_generator,
        steps_per_epoch=61,
        epochs=5,
        validation_data=test_generator,
        validation_steps=7)

Epoch 1/5
 5/61 [=>............................] - ETA: 11:19 - loss: 67.1586 - accuracy: 0.0469

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (112666125 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


 9/61 [===>..........................] - ETA: 11:33 - loss: 38.1725 - accuracy: 0.1458

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (108753612 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


10/61 [===>..........................] - ETA: 11:31 - loss: 34.5487 - accuracy: 0.1586

...

54/61 [=========================>....] - ETA: 2:21 - loss: 8.0367 - accuracy: 0.4119

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (154737000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


57/61 [===========================>..] - ETA: 1:19 - loss: 7.7106 - accuracy: 0.4147

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (89505558 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


61/61 [==============================] - 1207s 20s/step - loss: 7.3259 - accuracy: 0.4199 - val_loss: 1.9369 - val_accuracy: 0.2286
Epoch 2/5
61/61 [==============================] - 1614s 26s/step - loss: 1.8685 - accuracy: 0.5080 - val_loss: 1.9248 - val_accuracy: 0.2571
Epoch 3/5
61/61 [==============================] - 872s 14s/step - loss: 1.8062 - accuracy: 0.5069 - val_loss: 1.9396 - val_accuracy: 0.1857
Epoch 4/5
61/61 [==============================] - 897s 15s/step - loss: 1.7256 - accuracy: 0.5141 - val_loss: 1.9707 - val_accuracy: 0.1714
Epoch 5/5
61/61 [==============================] - 1283s 21s/step - loss: 1.5594 - accuracy: 0.5239 - val_loss: 2.1400 - val_accuracy: 0.2429


In [12]:
test_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6,
       6, 6, 6, 6, 6, 6], dtype=int32)

In [17]:
np.argmax(model.predict(test_generator), axis=1)

array([1, 1, 4, 1, 4, 4, 4, 3, 1, 1, 4, 1, 1, 1, 1, 4, 1, 4, 1, 4, 1, 1,
       4, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 3, 4, 3, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 4,
       1, 1, 4, 1, 4, 1])

In [18]:
confusion_matrix(test_generator.classes, np.argmax(model.predict(test_generator), axis=1))

array([[ 0,  3,  0,  0,  4,  0,  0],
       [ 0, 11,  0,  0,  5,  0,  0],
       [ 0,  6,  0,  1,  1,  0,  0],
       [ 0, 11,  0,  0,  4,  0,  0],
       [ 0,  6,  0,  1,  2,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0],
       [ 0,  4,  0,  1,  4,  0,  0]])